# 提示和提示模板
* 在与LLM的对话中引入编程。

## 介绍
* 输入：我们发送给LLM的提示。
* 输出：LLM的响应。
* 我们可以切换LLM并使用多个不同的LLM。

## 目录
* 大型语言模型（LLMs）。
* 提示和提示模板。
* 提示类型：零样本和少样本提示。
* 序列化：保存和加载提示。
* 解析输出。

## LangChain将LLM分为两种类型
1. LLM模型：文本补全模型。
2. 聊天模型：通过一系列消息进行对话，并可以定义特定的角色（系统提示）。这种类型已成为LangChain中最常用的。

## 查看差异  
* 即使有时 LangChain 文档对此可能会令人困惑，事实是文本补全模型和聊天模型都是大型语言模型（LLM）。  
* 但是，正如您在这个[playground](https://platform.openai.com/playground/chat?models=gpt-4o)中看到的，它们有一些显著的差异。请注意，LangChain 中的聊天模型有系统消息、人类消息（OpenAI 称之为“用户消息”）和 AI 消息（OpenAI 称之为“助手消息”）。  
* 自从 chatGPT 发布以来，聊天模型成为了最受欢迎的 LLM 类型，并被大多数 LLM 应用程序使用。

## 可以与 LangChain 一起使用的 LLM 列表
* 在 [这里](https://python.langchain.com/docs/integrations/llms/) 查看列表。

## 跟踪操作
从现在开始，我们可以从 LangSmith 跟踪此项目的操作 **和成本**：
* [smith.langchain.com](https://smith.langchain.com)

## 连接到位于此笔记本同一目录的 .env 文件

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

* 注意：由于目前市场上最佳的语言模型是 OpenAI，我们将默认使用它。您将在下一课中看到如何连接其他开源语言模型，如 Llama3 或 Mistral。

## LLM模型
* chatGPT-4发布前的趋势。
* 请参阅关于LLM模型的LangChain文档 [这里](https://python.langchain.com/v0.1/docs/modules/model_io/llms/)。

In [5]:
from langchain_openai import OpenAI

llmModel = OpenAI()

In [6]:
from langchain_openai import ChatOpenAI

chatModel = ChatOpenAI(model="gpt-3.5-turbo-0125")

## 提示和提示模板
**提示**是我们提供给一个语言模型的输入。这个输入将引导语言模型的响应方式。
有许多类型的提示：
* 简单指令。
* 带有一些示例的指令（少量示例）。
* 针对特定任务的具体上下文和问题。
* 等等。
* 请参阅 LangChain 文档中的提示 [这里](https://python.langchain.com/v0.1/docs/modules/model_io/prompts/quick_start/)。

**提示模板**是预定义的提示公式，通常需要一些额外的元素才能完整。这些额外的元素是用户提供的变量。
* 提示模板：当我们想要使用包含变量和其他元素的复杂提示时。一个提示模板可以包括：
    * 指令，
    * 少量示例，
    * 以及适合特定任务的具体上下文和问题。

In [1]:
from langchain_core.prompts import PromptTemplate

# prompt_template => prompt
prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} story about {topic}."
)

llmModelPrompt = prompt_template.format(
    adjective="curious", 
    topic="the Kennedy family"
)

llmModel.invoke(llmModelPrompt)

NameError: name 'llmModel' is not defined

In [8]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an {profession} expert on {topic}."),
        ("human", "Hello, Mr. {profession}, can you please answer a question?"),
        ("ai", "Sure!"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(
    profession="Historian",
    topic="The Kennedy family",
    user_input="How many grandchildren had Joseph P. Kennedy?"
)

response = chatModel.invoke(messages)

In [9]:
response

AIMessage(content='Joseph P. Kennedy, Sr. and his wife Rose Fitzgerald Kennedy had a total of nine grandchildren. Their children were John F. Kennedy, Robert F. Kennedy, Ted Kennedy, Eunice Kennedy Shriver, Patricia Kennedy Lawford, Jean Kennedy Smith, and Rosemary Kennedy.', response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 55, 'total_tokens': 112}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-970c06fd-afd3-4b21-8e9e-f51e97ab6511-0', usage_metadata={'input_tokens': 55, 'output_tokens': 57, 'total_tokens': 112})

In [10]:
print(response)

content='Joseph P. Kennedy, Sr. and his wife Rose Fitzgerald Kennedy had a total of nine grandchildren. Their children were John F. Kennedy, Robert F. Kennedy, Ted Kennedy, Eunice Kennedy Shriver, Patricia Kennedy Lawford, Jean Kennedy Smith, and Rosemary Kennedy.' response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 55, 'total_tokens': 112}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-970c06fd-afd3-4b21-8e9e-f51e97ab6511-0' usage_metadata={'input_tokens': 55, 'output_tokens': 57, 'total_tokens': 112}


In [11]:
print(response.content)

Joseph P. Kennedy, Sr. and his wife Rose Fitzgerald Kennedy had a total of nine grandchildren. Their children were John F. Kennedy, Robert F. Kennedy, Ted Kennedy, Eunice Kennedy Shriver, Patricia Kennedy Lawford, Jean Kennedy Smith, and Rosemary Kennedy.


#### Old way:

In [12]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import HumanMessagePromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are an Historian expert on the Kennedy family."
            )
        ),
        HumanMessagePromptTemplate.from_template("{user_input}"),
    ]
)

messages = chat_template.format_messages(
    user_input="Name the children and grandchildren of Joseph P. Kennedy?"
)

response = chatModel.invoke(messages)

In [13]:
print(response.content)

Joseph P. Kennedy and his wife Rose Fitzgerald Kennedy had nine children:

1. Joseph P. Kennedy Jr.
2. John F. Kennedy
3. Rosemary Kennedy
4. Kathleen Kennedy
5. Eunice Kennedy
6. Patricia Kennedy
7. Robert F. Kennedy
8. Jean Kennedy
9. Edward M. Kennedy

The grandchildren of Joseph P. Kennedy include Caroline Kennedy (daughter of John F. Kennedy), Maria Shriver (daughter of Eunice Kennedy), and Robert F. Kennedy Jr. (son of Robert F. Kennedy), among others.


#### ChatPromptTemplate的全部潜力是什么？
* 请查看LangChain API中的[对应页面](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html)。

## 基本提示策略

* 零样本提示：“对这条评论的情感进行分类：…”
* 少样本提示：“根据这些示例对这条评论的情感进行分类：…”
* 思维链提示：“根据这些示例及其背后推理的解释对这条评论的情感进行分类：…”

## Few Shot Prompting

In [27]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

In [28]:
examples = [
    {"input": "hi!", "output": "¡hola!"},
    {"input": "bye!", "output": "¡adiós!"},
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an English-Spanish translator."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

## How to execute the code from Visual Studio Code
* In Visual Studio Code, see the file 001-connect-llms.py
* In terminal, make sure you are in the directory of the file and run:
    * python 003-prompt-templates.py